In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
from openai import OpenAI

client = OpenAI()

In [7]:
import instructor

In [9]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'user', 'content': 'sup'}
    ]
)

response.choices[0].message.content

'Hey! Not much, just here to help. What’s up with you?'

In [10]:
#Adding instructor
instructor_client = instructor.from_openai(client)

In [11]:
from pydantic import BaseModel

class Greeting(BaseModel):
    hello: str

In [15]:
response = instructor_client.chat.completions.create(
    model='gpt-4o-mini',
    messages = [{
        "role": "user", 
        "content": 'sup'
    }], 
    response_model=Greeting
)

response

Greeting(hello='sup')

In [16]:
from pydantic import Field 
from datetime import date 
from typing import List 

class Person(BaseModel):
    name: str

class CalenderEvent(BaseModel):
    name: str 

    date: date
    participants: List[Person]

    address_number: str 
    street_name: str
    city_name: str 

    state_code: str = Field(pattern=r'[A-Z]{2}') #two letter state_code
    zip_code: str = Field(pattern=r'\d{5}') # 5 digits zip code

In [17]:
event_description = """
Alice and Bob are going to a science fair on Friday, January 2024.
The science fair is hosted at the gymnasium of Hazeldale Elementary
School at 20080 SW Farmington Road in Beaverton Oregon.
"""

In [18]:
def generate(
    response_model,
    user_prompt, 
    system_prompt="Your are a helpful assistant",
    model="gpt-4o-mini",
    max_retries=3
):
    event = instructor_client.chat.completions.create(
        model=model,
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt},
        ],
        response_model=response_model,
        max_retries=max_retries
    )  

    return event 

In [19]:
system_prompt = """ 
Make a calendar event. Respond in JSON with
the event name, date, list of participants,
and the address.
"""

user_prompt = "Event Description: "+ event_description
event = generate(
    CalenderEvent,
    user_prompt,
    system_prompt
)

In [20]:
event

CalenderEvent(name='Science Fair', date=datetime.date(2024, 1, 5), participants=[Person(name='Alice'), Person(name='Bob')], address_number='20080', street_name='SW Farmington Road', city_name='Beaverton', state_code='OR', zip_code='97007')

Retries

In [30]:
import os

print("Files in current directory:", os.listdir())

Files in current directory: ['L2_Structure_Outptut_BaseModel.ipynb', 'L3_Retry_based_Structure_Output.ipynb', 'utils.py']


In [38]:
from custom_utils import UsageTracker


instructor_client.clear("completion:response") # duplicate writes to the usage tracker.

tracker = UsageTracker()

# Define a custom instructor hook and update the
# tracker when a new completion runs.
def log_completion_kwargs(*args, **kwargs):
    usage = args[0].usage
    tracker.track(usage)

# run each time the server returns a chat completion to us.
instructor_client.on("completion:response", log_completion_kwargs)


In [39]:
# Clear the tracker before we run the completion
# so we arent' tracking multiple jobs.
tracker.clear()


event = generate(
    CalenderEvent,
    user_prompt, 
    system_prompt=system_prompt,
    max_retries=4,
)

event

CalenderEvent(name='Science Fair', date=datetime.date(2024, 1, 5), participants=[Person(name='Alice'), Person(name='Bob')], address_number='20080', street_name='SW Farmington Road', city_name='Beaverton', state_code='OR', zip_code='97007')

In [40]:
print("Input tokens:  ", tracker.input_tokens)
print("Output tokens: ", tracker.output_tokens)
print("Total tokens:  ", sum(tracker.total_tokens))
print("Num retries:   ", len(tracker.output_tokens))

Input tokens:   [235]
Output tokens:  [60]
Total tokens:   295
Num retries:    1


When retry method fails

In [41]:
from typing import Literal

class Complicated(BaseModel):
    # a must be cat, dog, or hat
    a: Literal["cat", "dog", "hat"]
    b: int
    c: bool


In [54]:
# Clear the tracker before we run the completion
# so we arent' tracking multiple jobs.
tracker.clear()

try:
    event = generate(
        Complicated, 
        "Write me a short essay on Dolly Parton.", 
        system_prompt="Don't generate anyting",
        max_retries=3,
    )

    # Show the event
    print(event)
except instructor.exceptions.InstructorRetryException as e : 
    print("We failed to parse!")
except Exception as e:
    raise e

a='hat' b=5 c=True


In [55]:
print("Input tokens:  ", tracker.input_tokens)
print("Output tokens: ", tracker.output_tokens)
print("Total tokens:  ", sum(tracker.total_tokens))
print("Num retries:   ", len(tracker.output_tokens))

Input tokens:   [101]
Output tokens:  [14]
Total tokens:   115
Num retries:    1
